In [2]:
import datetime as dt
import numpy as np
import pandas as pd
import requests
import json
from pprint import pprint
import csv
# from api_key import api_key
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [3]:
export_url = (f"https://api.census.gov/data/timeseries/intltrade/exports")
import_url = (f"https://api.census.gov/data/timeseries/intltrade/imports")
china_code = "5700"
usa_code = "1000"
month = []
year = []

In [4]:
export_by_commodity = requests.get(f"{export_url}/hs?get=E_COMMODITY,E_COMMODITY_SDESC,ALL_VAL_MO,ALL_VAL_YR&time=from+2015-01&COMM_LVL=HS2&CTY_CODE={china_code}").json()


In [5]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 4745:
    count += 1
    movalue.append(export_by_commodity[count][2])
    yrvalue.append(export_by_commodity[count][3])
    desc.append(export_by_commodity[count][1])
    period.append(export_by_commodity[count][4])
    code.append(export_by_commodity[count][0])
china_data_export = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value " : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [7]:
import_by_commodity = requests.get(f"{import_url}/hs?get=I_COMMODITY,I_COMMODITY_SDESC,GEN_VAL_MO,GEN_VAL_YR&time=from+2015-01&COMM_LVL=HS2&CTY_CODE={china_code}").json()


In [8]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 4802:
    count += 1
    movalue.append(import_by_commodity[count][2])
    yrvalue.append(import_by_commodity[count][3])
    desc.append(import_by_commodity[count][1])
    period.append(import_by_commodity[count][4])
    code.append(import_by_commodity[count][0])
china_data_import = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value " : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [9]:
china_data_export["YTD Value"] =pd.to_numeric(china_data_export["YTD Value"])

In [10]:
first_2015 = china_data_export[china_data_export["Period"].str.contains("2015")]

In [11]:
data_2015 = first_2015.groupby(["Comm Code","Commodity"])["YTD Value"].sum()

In [13]:
test= pd.DataFrame({"total" : data_2015})

In [32]:
data_2015= test.nlargest(10,"total")


In [34]:
data_2015 = data_2015.reset_index()

In [35]:
data_2015= data_2015.nlargest(10,"total")
data_2015

,Comm Code,Commodity,total
0,88,"AIRCRAFT, SPACECRAFT, AND PARTS THEREOF",95362331288
1,84,"NUCLEAR REACTORS, BOILERS, MACHINERY ETC.; PARTS",81687836243
2,85,ELECTRIC MACHINERY ETC; SOUND,80965955011
3,87,"VEHICLES, EXCEPT RAILWAY OR TRAMWAY, AND PARTS...",68852528392
4,12,"OIL SEEDS ETC.; MISC GRAIN, SEED, FRUIT, PLANT...",59289835223
5,90,"OPTIC, PHOTO ETC, MEDIC OR S",50091151820
6,39,PLASTICS AND ARTICLES THEREOF,31971602888
7,47,WOOD PULP ETC; RECOVD (WASTE & SCRAP) PPR & PPRBD,22330949053
8,10,CEREALS,16486726063
9,29,ORGANIC CHEMICALS,16250282113


In [36]:
data_2015 = data_2015.to_csv("data_2015.csv", index=False)